In [2]:
!pip install dash
!pip install dash_bootstrap_components
!pip install gensim
!pip install pyLDAvis
!pip install dash
!pip install dash_bootstrap_components
!pip install pyngrok
!pip install bertopic top2vec[torch] umap-learn hdbscan
!pip install contractions
!pip install spacy
!pip install --upgrade numpy
!pip install --upgrade gensim
!python -m spacy download en_core_web_sm

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  A

In [3]:
!pip install dash_table

In [1]:
# dashboard_app.py
# ----------------------------------------------------------
# Dash dashboard for Fake-News project with clustering
# ----------------------------------------------------------

import pickle
import numpy as np
import pandas as pd
from pathlib import Path

import dash
from dash import html, dcc, Input, Output, State, dash_table
import dash_bootstrap_components as dbc

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity



#  Utilidades estándar

import os
import re
import ast
import pickle
from pathlib import Path
from collections import Counter


#  Manipulación de datos

import numpy as np
import pandas as pd


#  Visualización

import matplotlib.pyplot as plt
import seaborn as sns


#  Progreso y entorno

from tqdm.notebook import tqdm
tqdm.pandas()


#  Web scraping / Limpieza de HTML

from bs4 import BeautifulSoup


#  NLP y procesamiento de texto

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import contractions

# Verificar y descargar paquetes NLTK

def check_nltk_packages():
    packages = ['punkt', 'stopwords', 'omw-1.4', 'wordnet']
    for package in packages:
        try:
            nltk.data.find(f'tokenizers/{package}')
        except LookupError:
            nltk.download(package)
check_nltk_packages()


#  Vectorización y modelos de texto

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Doc2Vec, TfidfModel, CoherenceModel, LdaModel
from gensim.models.doc2vec import TaggedDocument
from gensim.models.phrases import Phrases
from gensim.corpora import Dictionary
import gensim.downloader as api
from gensim.matutils import corpus2dense
import pyLDAvis
import pyLDAvis.gensim_models
from bertopic import BERTopic
from top2vec import Top2Vec
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE
import umap.umap_ as umap

#  Preprocesamiento y normalización

from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


#  Modelos de clasificación

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


#  Métricas de evaluación

from sklearn.metrics import (
    classification_report, accuracy_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve,
    silhouette_score, davies_bouldin_score,
    adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score
)


#  Reducción de dimensionalidad

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP


#  Clustering

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering


#  Similitud

from sklearn.metrics.pairwise import cosine_similarity


#  Dash - Visualización Web

import dash
from dash import html, dcc, Input, Output, State, dash_table
import dash_bootstrap_components as dbc


#  Otros

from sklearn.utils import resample

# Semilla global
seed = 10

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.11/dist-packages/hdbscan/plots.py:448: DeprecationWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.11/dist-packages/hdbscan/robust_single_linkage_.py:154: DeprecationWarning: invalid escape sequence '\{'
  """Perform robust single linkage clustering from a vector array
/usr/local/lib/python3.11/dist-packages/geopandas/_compat.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).
  import shapely.geos
/usr/local/

In [4]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import dash_table
import joblib
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
import contractions
import plotly.graph_objects as go
import gensim.downloader as api

<ipython-input-4-68eab848db0e>:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [35]:
df_total.head()

,title,text,target,text_length,title_length,word_count,word_count_title,text_tokens,title_tokens
0,WATCH: Republicans Disrespectfully Shout Down...,Democrats stood up for the American people but...,1,2214,106,354,14,"['democrat', 'stand', 'american', 'people', 'r...","['watch', 'republican', 'disrespectfully', 'sh..."
1,Saudi Arabia says U.N. report on Yemen 'inaccu...,UNITED NATIONS (Reuters) - Saudi Arabia on Fri...,0,2531,66,396,10,"['united', 'nation', 'reuters', 'saudi', 'arab...","['saudi', 'arabia', 'say', 'report', 'yemen', ..."
2,Watch Morning Joe’s Mika DESTROY Paul Ryan Fo...,Republican House Speaker Paul Ryan has been ge...,1,2107,86,381,14,"['republican', 'house', 'speaker', 'paul', 'ry...","['watch', 'morning', 'joe', 'mika', 'destroy',..."
3,House Democrats mull Pelosi leadership future,WASHINGTON (Reuters) - U.S. House Democratic L...,0,3778,45,618,6,"['washington', 'reuters', 'house', 'democratic...","['house', 'democrat', 'mull', 'pelosi', 'leade..."
4,BREAKING: MI Court Of Appeals Orders Vote Reco...,The Michigan Court of Appeals rejected Green P...,1,4473,119,745,19,"['michigan', 'court', 'appeal', 'reject', 'gre...","['breaking', 'mi', 'court', 'appeal', 'order',..."


In [15]:
!pip install jupyter-dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.7 MB/s eta 0:00:00


In [43]:
# ───────────────────────────────────────────────────────────────
# Imports and Setup
# ───────────────────────────────────────────────────────────────
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import joblib
import spacy
import contractions
import gensim.downloader as api
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# ───────────────────────────────────────────────────────────────
# Load models and data
# ───────────────────────────────────────────────────────────────
model_titles = joblib.load("model_titles.pkl")
model_all = joblib.load("model_all.pkl")
glove_model = api.load("glove-wiki-gigaword-50")
model_doc2vec = joblib.load("doc2vec_model_text.model")
df_total = pd.read_csv("df_total.csv")
kmeans_titles = joblib.load("kmeans_titles.pkl")
lda_model = joblib.load("model_lda.gensim")

nlp = spacy.load("en_core_web_sm")
wnl = WordNetLemmatizer()

# ───────────────────────────────────────────────────────────────
# Text preprocessing
# ───────────────────────────────────────────────────────────────
broken_contractions = {
    r"\bdon t\b": "don't", r"\bwon t\b": "won't", r"\bcan t\b": "can't",
    r"\bi m\b": "I'm", r"\byou re\b": "you're", r"\bhe s\b": "he's",
    r"\bshe s\b": "she's", r"\bit s\b": "it's", r"\bwe re\b": "we're",
    r"\bthey re\b": "they're", r"\bthat s\b": "that's", r"\bthere s\b": "there's",
    r"\bwhat s\b": "what's", r"\s+s\s+": "'s ", r"\.(?![\s.])": ". "
}

def fix_broken_contractions(text):
    for pattern, replacement in broken_contractions.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

def text_preprocessing_pipeline(text: str):
    text = BeautifulSoup(text, 'lxml').get_text()
    text = fix_broken_contractions(text)
    text = contractions.fix(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc
              if not token.is_stop and not token.is_punct and token.lemma_.isalpha()]
    return [wnl.lemmatize(t) for t in tokens if t.isalnum()]

def glove_vector(text: str):
    tokens = text_preprocessing_pipeline(text)
    vecs = [glove_model[w] for w in tokens if w in glove_model]
    return np.mean(vecs, axis=0) if vecs else np.zeros(glove_model.vector_size)

def doc2vec_vector(text: str):
    tokens = text_preprocessing_pipeline(text)
    return model_doc2vec.infer_vector(tokens)

def prob_bar(p: float, fake: bool):
    color = "#dc3545" if fake else "#198754"
    fig = go.Figure(go.Bar(
        x=[p], y=[""],
        orientation="h",
        marker=dict(color=color),
        text=f"{p*100:.1f} %", textposition="outside"
    ))
    fig.update_layout(
        xaxis=dict(range=[0, 1], showticklabels=False),
        yaxis=dict(showticklabels=False),
        margin=dict(l=0, r=0, t=0, b=0),
        height=80
    )
    return fig

# ───────────────────────────────────────────────────────────────
# Cluster and topic options
# ───────────────────────────────────────────────────────────────
# Predict clusters for all titles using the KMeans model
title_vectors = np.vstack([glove_vector(t) for t in df_total['title']])
df_total['kmeans_title_cluster'] = kmeans_titles.predict(title_vectors)

# Now generate the cluster names
cluster_ids = sorted(np.unique(df_total['kmeans_title_cluster']))
cluster_names = [f"Cluster {i}" for i in cluster_ids]
topic_names = [f"Topic {i}" for i in range(lda_model.num_topics)]

# ───────────────────────────────────────────────────────────────
# App Initialization
# ───────────────────────────────────────────────────────────────
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                title="Fake News Dashboard")

navbar = dbc.Navbar(
    dbc.Container([dbc.NavbarBrand("📰 Fake-News Insight Hub", className="fw-bold")]),
    color="dark", dark=True, className="mb-4"
)

# ───────────────────────────────────────────────────────────────
# Tab Layouts
# ───────────────────────────────────────────────────────────────
tab_classifier = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.RadioItems(
                id="mode-selector",
                inline=True,
                className="mb-2",
                options=[
                    {"label": "Full article", "value": "full"},
                    {"label": "Title only",  "value": "title"},
                ],
                value="full"
            ),
            dcc.Textarea(
                id="input-article",
                placeholder="Paste or type here…",
                style={"width": "100%", "height": 200}
            ),
            dbc.Button("Analyze", id="btn-analyze", color="primary", className="mt-2"),
            html.Div(id="pred-label", className="mt-3 fw-bold fs-4"),
            dcc.Graph(id="pred-prob-bar", config={"displayModeBar": False}),
            html.Div(id="cluster-output", className="mt-1")
        ], lg=6),
        dbc.Col([
            html.H5("Most similar REAL news"),
            dash_table.DataTable(
                id="similar-real-table",
                columns=[{"name": "Title", "id": "title"}],
                style_cell={"whiteSpace": "normal"},
                style_table={"height": "200px", "overflowY": "auto"}
            ),
            html.H5("Related articles in same cluster", className="mt-3"),
            dash_table.DataTable(
                id="related-cluster-table",
                columns=[
                    {"name": "Title", "id": "title"},
                    {"name": "Real/Fake", "id": "flag"}
                ],
                style_cell={"whiteSpace": "normal"},
                style_table={"height": "200px", "overflowY": "auto"}
            )
        ], lg=6)
    ])
], fluid=True)

tab_cluster_finder = dbc.Container([
    dbc.Row([dbc.Col([
        dcc.Textarea(
            id="input-cluster",
            placeholder="Paste text to locate its cluster…",
            style={"width": "100%", "height": 200}
        ),
        dbc.Button("Find cluster", id="btn-find-cluster", color="secondary", className="mt-2"),
        html.Div(id="cluster-only-result", className="mt-3 fw-bold fs-4")
    ], lg=6)])
], fluid=True)

tab_true_news = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Topic"),
        dcc.Dropdown(
            id="topic-selector-true",
            options=[{"label": n, "value": i} for i, n in enumerate(topic_names)],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="true-news-table",
        columns=[{"name": "Title", "id": "title"}],
        style_cell={"whiteSpace": "normal"},
        style_table={"height": "500px", "overflowY": "auto"}
    )
], fluid=True)

tab_fake_news = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Topic"),
        dcc.Dropdown(
            id="topic-selector-fake",
            options=[{"label": n, "value": i} for i, n in enumerate(topic_names)],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="fake-news-table",
        columns=[{"name": "Title", "id": "title"}],
        style_cell={"whiteSpace": "normal"},
        style_table={"height": "500px", "overflowY": "auto"}
    )
], fluid=True)

tab_clusters = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Cluster"),
        dcc.Dropdown(
            id="cluster-selector",
            options=[{"label": name, "value": int(name.split()[-1])} for name in cluster_names],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="cluster-news-table",
        columns=[
            {"name": "Title", "id": "title"},
            {"name": "Real/Fake", "id": "flag"}
        ],
        style_cell={"whiteSpace": "normal"},
        style_table={"height": "500px", "overflowY": "auto"}
    )
], fluid=True)

tabs = dcc.Tabs(id="tabs", value="tab-classifier", children=[
    dcc.Tab(label="🔍 Classify & Suggest", value="tab-classifier", children=tab_classifier),
    dcc.Tab(label="📂 Cluster Finder", value="tab-cluster-finder", children=tab_cluster_finder),
    dcc.Tab(label="✅ True News by Topic", value="tab-true", children=tab_true_news),
    dcc.Tab(label="🚫 Fake News by Topic", value="tab-fake", children=tab_fake_news),
    dcc.Tab(label="🗂 Explore by Cluster", value="tab-clusters", children=tab_clusters),
])

app.layout = html.Div([navbar, tabs])

# ───────────────────────────────────────────────────────────────
# Callbacks
# ───────────────────────────────────────────────────────────────
@app.callback(
    Output("pred-label", "children"),
    Output("pred-prob-bar", "figure"),
    Output("related-cluster-table", "data"),
    Input("btn-analyze", "n_clicks"),
    State("input-article", "value"),
    State("mode-selector", "value"),
    prevent_initial_call=True
)
def classify_article(_, text, mode):
    if not text or text.strip() == "":
        return "Please provide text.", go.Figure(), []

    if mode == "title":
        vec = glove_vector(text).reshape(1, -1).astype(np.float64)
        model = model_titles
        cluster = int(kmeans_titles.predict(vec)[0])
    else:
        vec_title = glove_vector(text)
        vec_text = doc2vec_vector(text)
        vec = vec_title.reshape(1, -1).astype(np.float64)
        model = model_all
        sim_row = df_total[df_total.text == text]
        cluster = int(sim_row["clustering_title"].values[0]) if not sim_row.empty else -1

    label = model.predict(vec)[0]
    try:
        proba = model.predict_proba(vec)[0, int(label)]
    except:
        proba = 1.0

    pred_txt = "🔴 FAKE news" if label else "🟢 REAL news"
    fig = prob_bar(proba, fake=bool(label))

    if cluster == -1:
        related_data = []
    else:
        related = df_total[df_total.clustering_title == cluster].copy()
        related["flag"] = related["target"].map({0: "Real", 1: "Fake"})
        related_data = related[["title", "flag"]].head(10).to_dict("records")

    return pred_txt, fig, related_data

# ───────────────────────────────────────────────────────────────
# Run Server
# ───────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run(port=8050, debug=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
<ipython-input-43-6e92b92cea38>:52: MarkupResemblesLocatorWarning:

The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import 

ValueError: Buffer dtype mismatch, expected 'const float' but got 'double'

In [39]:
import joblib
model_titles = joblib.load("model_titles.pkl")
model_all = joblib.load("model_all.pkl")
glove_model = api.load("glove-wiki-gigaword-50")
model_doc2vec = joblib.load("doc2vec_model_text.model")
df_total = pd.read_csv("df_total.csv")
kmeans_titles = joblib.load("kmeans_titles.pkl")
lda_model = joblib.load("lda_model.gensim ")


nlp = spacy.load("en_core_web_sm")
wnl = WordNetLemmatizer()

broken_contractions = {
    r"\bdon t\b": "don't", r"\bwon t\b": "won't", r"\bcan t\b": "can't",
    r"\bi m\b": "I'm", r"\byou re\b": "you're", r"\bhe s\b": "he's",
    r"\bshe s\b": "she's", r"\bit s\b": "it's", r"\bwe re\b": "we're",
    r"\bthey re\b": "they're", r"\bthat s\b": "that's", r"\bthere s\b": "there's",
    r"\bwhat s\b": "what's", r"\s+s\s+": "'s ", r"\.(?![\s.])": ". "
}

def fix_broken_contractions(text):
    for pattern, replacement in broken_contractions.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

def text_preprocessing_pipeline(text: str):
    text = BeautifulSoup(text, 'lxml').get_text()
    text = fix_broken_contractions(text)
    text = contractions.fix(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc
              if not token.is_stop and not token.is_punct and token.lemma_.isalpha()]
    return [wnl.lemmatize(t) for t in tokens if t.isalnum()]

def glove_vector(text: str):
    tokens = text_preprocessing_pipeline(text)
    vecs = [glove_model[w] for w in tokens if w in glove_model]
    return np.mean(vecs, axis=0) if vecs else np.zeros(glove_model.vector_size)

def doc2vec_vector(text: str):
    tokens = text_preprocessing_pipeline(text)
    return model_doc2vec.infer_vector(tokens)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 27932

In [6]:
def prob_bar(p: float, fake: bool):
    color = "#dc3545" if fake else "#198754"
    fig = go.Figure(go.Bar(
        x=[p], y=[""],
        orientation="h",
        marker=dict(color=color),
        text=f"{p*100:.1f} %", textposition="outside"
    ))
    fig.update_layout(
        xaxis=dict(range=[0, 1], showticklabels=False),
        yaxis=dict(showticklabels=False),
        margin=dict(l=0, r=0, t=0, b=0),
        height=80
    )
    return fig

In [38]:
# Create app
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import joblib

# ──────────────────────────────────────────────────────────
# 3.  DASH APP LAYOUT
# ──────────────────────────────────────────────────────────
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                title="Fake News Dashboard")

# ─ Navbar ────────────────────────────────────────────────
navbar = dbc.Navbar(
    dbc.Container([
        dbc.NavbarBrand("📰 Fake-News Insight Hub", className="fw-bold")
    ]),
    color="dark", dark=True, className="mb-4"
)

# ──────────────────────────────────────────────────────────
# utilities
# ──────────────────────────────────────────────────────────
def prob_bar(p: float, fake: bool):
    """
    Return a Plotly bar chart figure showing a horizontal
    confidence bar, coloured red for fake (True) or green for real (False).
    """
    import plotly.graph_objects as go
    color = "#dc3545" if fake else "#198754"
    fig = go.Figure(go.Bar(
        x=[p], y=[""],
        orientation="h",
        marker=dict(color=color),
        text=f"{p*100:.1f} %", textposition="outside"
    ))
    fig.update_layout(
        xaxis=dict(range=[0, 1], showticklabels=False),
        yaxis=dict(showticklabels=False),
        margin=dict(l=0, r=0, t=0, b=0),
        height=80
    )
    return fig

# ──────────────────────────────────────────────────────────
# Tab 1 – Classify & Suggest
# ──────────────────────────────────────────────────────────
tab_classifier = dbc.Container([
    dbc.Row([

        # Left column: choose mode, input, analyze, results
        dbc.Col([
            dbc.RadioItems(
                id="mode-selector",
                inline=True,
                className="mb-2",
                options=[
                    {"label": "Full article", "value": "full"},
                    {"label": "Title only",  "value": "title"},
                ],
                value="full"
            ),
            dcc.Textarea(
                id="input-article",
                placeholder="Paste or type here…",
                style={"width": "100%", "height": 200}
            ),
            dbc.Button("Analyze", id="btn-analyze", color="primary", className="mt-2"),
            html.Div(id="pred-label", className="mt-3 fw-bold fs-4"),
            dcc.Graph(id="pred-prob-bar", config={"displayModeBar": False}),
            html.Div(id="cluster-output", className="mt-1")
        ], lg=6),

        # Right column: tables
        dbc.Col([
            html.H5("Most similar REAL news"),
            dash_table.DataTable(
                id="similar-real-table",
                columns=[{"name": "Title", "id": "title"}],
                style_cell={"whiteSpace": "normal"},
                style_table={"height":"200px","overflowY":"auto"}
            ),
            html.H5("Related articles in same cluster", className="mt-3"),
            dash_table.DataTable(
                id="related-cluster-table",
                columns=[
                    {"name":"Title","id":"title"},
                    {"name":"Real/Fake","id":"flag"}
                ],
                style_cell={"whiteSpace": "normal"},
                style_table={"height":"200px","overflowY":"auto"}
            )
        ], lg=6)

    ])
], fluid=True)

# ──────────────────────────────────────────────────────────
# Tab 2 – Cluster Finder
# ──────────────────────────────────────────────────────────
tab_cluster_finder = dbc.Container([
    dbc.Row([dbc.Col([
        dcc.Textarea(
            id="input-cluster",
            placeholder="Paste text to locate its cluster…",
            style={"width":"100%","height":200}
        ),
        dbc.Button("Find cluster", id="btn-find-cluster", color="secondary", className="mt-2"),
        html.Div(id="cluster-only-result", className="mt-3 fw-bold fs-4")
    ], lg=6)])
], fluid=True)

# ──────────────────────────────────────────────────────────
# Tab 3 – True News by Topic
# ──────────────────────────────────────────────────────────
tab_true_news = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Topic"),
        dcc.Dropdown(
            id="topic-selector-true",
            options=[{"label": n, "value": i} for i, n in enumerate(topic_names)],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="true-news-table",
        columns=[{"name":"Title","id":"title"}],
        style_cell={"whiteSpace":"normal"},
        style_table={"height":"500px","overflowY":"auto"}
    )
], fluid=True)

# ──────────────────────────────────────────────────────────
# Tab 4 – Fake News by Topic
# ──────────────────────────────────────────────────────────
tab_fake_news = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Topic"),
        dcc.Dropdown(
            id="topic-selector-fake",
            options=[{"label": n, "value": i} for i, n in enumerate(topic_names)],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="fake-news-table",
        columns=[{"name":"Title","id":"title"}],
        style_cell={"whiteSpace":"normal"},
        style_table={"height":"500px","overflowY":"auto"}
    )
], fluid=True)

# ──────────────────────────────────────────────────────────
# Tab 5 – Explore by Cluster
# ──────────────────────────────────────────────────────────
tab_clusters = dbc.Container([
    dbc.Row([dbc.Col([
        html.H5("Select Cluster"),
        dcc.Dropdown(
            id="cluster-selector",
            options=[{"label": n, "value": i} for i, n in enumerate(cluster_names)],
            value=0
        )
    ], lg=3)]),
    dash_table.DataTable(
        id="cluster-news-table",
        columns=[
            {"name":"Title","id":"title"},
            {"name":"Real/Fake","id":"flag"}
        ],
        style_cell={"whiteSpace":"normal"},
        style_table={"height":"500px","overflowY":"auto"}
    )
], fluid=True)

# ──────────────────────────────────────────────────────────
# Assemble all tabs
# ──────────────────────────────────────────────────────────
tabs = dcc.Tabs(id="tabs", value="tab-classifier", children=[
    dcc.Tab(label="🔍 Classify & Suggest",   value="tab-classifier",       children=tab_classifier),
    dcc.Tab(label="📂 Cluster Finder",       value="tab-cluster-finder",   children=tab_cluster_finder),
    dcc.Tab(label="✅ True News by Topic",   value="tab-true",             children=tab_true_news),
    dcc.Tab(label="🚫 Fake News by Topic",   value="tab-fake",             children=tab_fake_news),
    dcc.Tab(label="🗂 Explore by Cluster",   value="tab-clusters",         children=tab_clusters),
])

app.layout = html.Div([navbar, tabs])



NameError: name 'topic_names' is not defined

In [ ]:
@app.callback(
    Output("pred-label", "children"),
    Output("pred-prob-bar", "figure"),
    Output("related-cluster-table", "data"),
    Input("btn-analyze", "n_clicks"),
    State("input-title", "value"),
    State("input-text", "value"),
    State("mode-selector", "value"),
    prevent_initial_call=True
)
def classify_article(_, title_text, full_text, mode):
    if mode == "title":
        if not title_text or title_text.strip() == "":
            return "Please enter a title.", go.Figure(), []
        vec = glove_vector(title_text).reshape(1, -1).astype(np.float64)
        model = model_titles
        cluster = int(kmeans_titles.predict(vec)[0])
    else:
        if not title_text or not full_text:
            return "Please provide both title and text.", go.Figure(), []
        vec_title = glove_vector(title_text)
        vec_text = doc2vec_vector(full_text)
        vec = glove_vector(title_text).reshape(1, -1).astype(np.float64)

        model = model_all
        cluster = int(df_total[df_total.title == title_text]['clustering_title'].values[0]) if title_text in df_total.title.values else -1

    label = model.predict(vec)[0]
    try:
        proba = model.predict_proba(vec)[0, label]
    except:
        proba = 1.0

    pred_txt = "🔴 FAKE news" if label else "🟢 REAL news"
    fig = prob_bar(proba, fake=bool(label))

    if cluster == -1:
        related_data = []
    else:
        related = df_total[df_total.clustering_title == cluster].copy()
        related["flag"] = related["target"].map({0: "Real", 1: "Fake"})
        related_data = related[["title", "flag"]].head(10).to_dict("records")

    return pred_txt, fig, related_data

if __name__ == "__main__":
    app.run(port=8050, debug=True)

In [34]:
# ──────────────────────────────────────────────────────────
# 5.  RUN SERVER
# ──────────────────────────────────────────────────────────
from pyngrok import ngrok
import threading

# Expone el puerto 8050
ngrok.set_auth_token("2wiWiUlcCP4LrngURMChCetHbi9_2XcAkjAz4nZ5q9fhPTQiq")

public_url = ngrok.connect(addr="8050")
print("Tu app está disponible en:", public_url)

# Ejecuta tu app Dash/Flask en segundo plano
def run_app():
    app.run(port=8050, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.start()

Tu app está disponible en: NgrokTunnel: "https://6f63-35-203-149-68.ngrok-free.app" -> "http://localhost:8050"


In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import dash_table
import joblib
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
import contractions
import plotly.graph_objects as go
import gensim.downloader as api

# ───────────────────────────────────────────────
# 1. LOAD MODELS AND UTILITIES
# ───────────────────────────────────────────────


def prob_bar(p: float, fake: bool):
    color = "#dc3545" if fake else "#198754"
    fig = go.Figure(go.Bar(
        x=[p], y=[""],
        orientation="h",
        marker=dict(color=color),
        text=f"{p*100:.1f} %", textposition="outside"
    ))
    fig.update_layout(
        xaxis=dict(range=[0, 1], showticklabels=False),
        yaxis=dict(showticklabels=False),
        margin=dict(l=0, r=0, t=0, b=0),
        height=80
    )
    return fig

# ───────────────────────────────────────────────
# 2. DASH LAYOUT
# ───────────────────────────────────────────────
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                title="Fake News Dashboard")

navbar = dbc.Navbar(
    dbc.Container([
        dbc.NavbarBrand("📰 Fake-News Insight Hub", className="fw-bold")
    ]),
    color="dark", dark=True, className="mb-4"
)

tab_classifier = dbc.Container([
    dbc.Row([

        dbc.Col([
            dbc.RadioItems(
                id="mode-selector",
                inline=True,
                className="mb-2",
                options=[
                    {"label": "Full article", "value": "full"},
                    {"label": "Title only",  "value": "title"},
                ],
                value="full"
            ),
            dcc.Textarea(
                id="input-article",
                placeholder="Paste or type here…",
                style={"width": "100%", "height": 200}
            ),
            dbc.Button("Analyze", id="btn-analyze", color="primary", className="mt-2"),
            html.Div(id="pred-label", className="mt-3 fw-bold fs-4"),
            dcc.Graph(id="pred-prob-bar", config={"displayModeBar": False}),
            html.Div(id="cluster-output", className="mt-1")
        ], lg=6),

        dbc.Col([
            html.H5("Most similar REAL news"),
            dash_table.DataTable(
                id="similar-real-table",
                columns=[{"name": "Title", "id": "title"}],
                style_cell={"whiteSpace": "normal"},
                style_table={"height":"200px","overflowY":"auto"}
            ),
            html.H5("Related articles in same cluster", className="mt-3"),
            dash_table.DataTable(
                id="related-cluster-table",
                columns=[
                    {"name":"Title","id":"title"},
                    {"name":"Real/Fake","id":"flag"}
                ],
                style_cell={"whiteSpace": "normal"},
                style_table={"height":"200px","overflowY":"auto"}
            )
        ], lg=6)

    ])
], fluid=True)

app.layout = html.Div([navbar,
    dcc.Tabs(id="tabs", value="tab-classifier", children=[
        dcc.Tab(label="🔍 Classify & Suggest", value="tab-classifier", children=tab_classifier)
    ])
])

# ───────────────────────────────────────────────
# 3. CALLBACKS
# ───────────────────────────────────────────────
@app.callback(
    Output("pred-label",      "children"),
    Output("pred-prob-bar",   "figure"),
    Output("cluster-output",  "children"),
    Output("similar-real-table",    "data"),
    Output("related-cluster-table", "data"),
    Input("btn-analyze", "n_clicks"),
    State("input-article", "value"),
    State("mode-selector",  "value"),
    prevent_initial_call=True
)
def run_classifier(_, raw_text, mode):
    if not raw_text:
        return (
            "Please input some text.",
            prob_bar(0.0, False),
            "",
            [],
            []
        )

    tokens = text_preprocessing_pipeline(raw_text)
    vec = np.mean([glove_model[w] for w in tokens if w in glove_model] or
                  [np.zeros(glove_model.vector_size)], axis=0).reshape(1, -1)

    if mode == "title":
        model = model_titles
    else:
        model = model_all

    label = model.predict(vec)[0]
    proba = model.predict_proba(vec)[0, label]
    pred_txt = "🔴 FAKE news" if label else "🟢 REAL news"
    fig = prob_bar(proba, fake=bool(label))

    return pred_txt, fig, "", [], []

# ───────────────────────────────────────────────
# 4. RUN APP
# ───────────────────────────────────────────────
if __name__ == "__main__":
    app.run(port=8050, debug=False)


Tu app está disponible en: NgrokTunnel: "https://714c-35-203-149-68.ngrok-free.app" -> "http://localhost:8050"
